In [1]:
import pandas as pd
#import italy covid cases
df_covid_cases = pd.read_csv('../datasets/dpc-covid19-ita-province.csv',delimiter=',', keep_default_na=False, na_values=[''])
print(df_covid_cases.head())
#import environmental air quality data
df_air_data = pd.read_csv('../datasets/waqi-covid19-airqualitydata-2020.csv',delimiter=',', keep_default_na=False, na_values=[''])
print(df_air_data.head())

                  data stato  codice_regione denominazione_regione  \
0  2020-02-24T18:00:00   ITA              13               Abruzzo   
1  2020-02-24T18:00:00   ITA              13               Abruzzo   
2  2020-02-24T18:00:00   ITA              13               Abruzzo   
3  2020-02-24T18:00:00   ITA              13               Abruzzo   
4  2020-02-24T18:00:00   ITA              13               Abruzzo   

   codice_provincia               denominazione_provincia sigla_provincia  \
0                66                              L'Aquila              AQ   
1                67                                Teramo              TE   
2                68                               Pescara              PE   
3                69                                Chieti              CH   
4               979  In fase di definizione/aggiornamento             NaN   

         lat       long  totale_casi note  
0  42.351222  13.398438            0  NaN  
1  42.658918  13.704400     

In [2]:
# cast operation
df_covid_cases.dtypes
df_covid_cases['data'] = pd.to_datetime(df_covid_cases['data'], format="%Y-%m-%dT%H:%M:%S")
df_covid_cases['data'] = df_covid_cases['data'].dt.date
print(df_covid_cases.head())
print(df_air_data.dtypes)
df_air_data['Date'] = pd.to_datetime(df_air_data['Date'], format="%Y-%m-%d")
print(df_air_data.dtypes)
print(df_air_data.head())

         data stato  codice_regione denominazione_regione  codice_provincia  \
0  2020-02-24   ITA              13               Abruzzo                66   
1  2020-02-24   ITA              13               Abruzzo                67   
2  2020-02-24   ITA              13               Abruzzo                68   
3  2020-02-24   ITA              13               Abruzzo                69   
4  2020-02-24   ITA              13               Abruzzo               979   

                denominazione_provincia sigla_provincia        lat       long  \
0                              L'Aquila              AQ  42.351222  13.398438   
1                                Teramo              TE  42.658918  13.704400   
2                               Pescara              PE  42.464584  14.213648   
3                                Chieti              CH  42.351032  14.167546   
4  In fase di definizione/aggiornamento             NaN        NaN        NaN   

   totale_casi note  
0            0  

In [3]:
import pandasql
# compute IAQ
# filter Italy
df_air_data = df_air_data.loc[df_air_data['Country'] == 'IT']
# loop for each city
cities = df_air_data.groupby(['City']).count().index
df_iqa = pd.DataFrame(columns=['Date', 'City', 'IQA'])

for c in cities:
    print(c) 
    # filter dataframe by city
    df_air_data_c = df_air_data.loc[df_air_data['City'] == c]
    # order by Date
    df_air_data_c = df_air_data_c.sort_values(by=['Date'])
    #filter species: pm10 & no2 & 03
    df_air_data_c = df_air_data_c.loc[df_air_data_c['Specie'].isin(['pm10','no2','o3'])]
    #print(df_air_data_c.loc[df_air_data_c['Specie'].isin(['pm10'])]['median'].head())
    df_air_data_c['Spm10'] = (df_air_data_c.loc[df_air_data_c['Specie'].isin(['pm10'])]['median']/50)*100
    df_air_data_c['So3'] = (df_air_data_c.loc[df_air_data_c['Specie'].isin(['o3'])]['median']/120)*100
    df_air_data_c['Sno2'] = (df_air_data_c.loc[df_air_data_c['Specie'].isin(['no2'])]['median']/200)*100
    #df_air_data_c = df_air_data_c.fillna(0)
    #print(df_air_data_c.head(3))
    df_air_data_c_g = pandasql.sqldf(
                    "select "
                    "Date, "
                    "City, "
                    "max(So3) as So3, "
                    "max(Sno2) as Sno2, "
                    "max(Spm10) as Spm10 "
                    "from df_air_data_c "
                    "group by Date,City ",locals())
    #print(df_air_data_c_g.head(10))
    df_air_data_c_g['IQA'] =  df_air_data_c_g[['Spm10','So3','Sno2']].max(axis=1)
    #print(df_air_data_c_g.head(10))
    df_air_data_c_g = df_air_data_c_g[['Date', 'City','IQA']]
    #print(df_air_data_c_g.head(10))
    frames = [df_iqa, df_air_data_c_g]
    result = pd.concat(frames)
    df_iqa = result
# cast operation
df_iqa['Date'] = pd.to_datetime(df_iqa['Date'], format="%Y-%m-%dT%H:%M:%S")
test = pandasql.sqldf(
                "select City,"
                "count(*) "
                "from df_iqa "
                "group by City"
                ,locals())
print(test)
print(df_iqa.head())

Bologna
Brescia
Florence
Livorno
Milan
Modena
Naples
Parma
Prato
Rome
Trieste
Turin
        City  count(*)
0    Bologna       215
1    Brescia       213
2   Florence       212
3    Livorno       213
4      Milan       213
5     Modena       215
6     Naples       216
7      Parma       215
8      Prato       210
9       Rome       214
10   Trieste       183
        Date     City   IQA
0 2019-12-30  Bologna  68.0
1 2019-12-31  Bologna  70.0
2 2020-01-01  Bologna  58.0
3 2020-01-02  Bologna  92.0
4 2020-01-03  Bologna  48.0


In [29]:
# cities target covid
df_covid_cases_t = df_covid_cases.loc[df_covid_cases['denominazione_provincia'].isin(['Napoli','Milano','Roma','Brescia','Bologna','Torino','Firenze'])]
#cast operation
df_covid_cases_t['data'] = pd.to_datetime(df_covid_cases_t['data'], format="%Y-%m-%d")
print(df_covid_cases_t.head())
# replace english to italian language
df_iqa.loc[df_iqa['City'] == 'Naples', 'City'] = 'Napoli'
df_iqa.loc[df_iqa['City'] == 'Milan', 'City'] = 'Milano'
df_iqa.loc[df_iqa['City'] == 'Rome', 'City'] = 'Roma'
df_iqa.loc[df_iqa['City'] == 'Florence', 'City'] = 'Firenze'
df_iqa.loc[df_iqa['City'] == 'Turin', 'City'] = 'Torino'
print(df_iqa.head())

# join operation
df_covid_iqa = pandasql.sqldf(
                    "select t1.*,"
                    "t2.IQA "
                    "from  df_covid_cases_t as t1 "
                    "join df_iqa as t2 "
                    "on t1.data = t2.Date and t1.denominazione_provincia = t2.City"
                    ,locals())
#cast datetime
df_covid_iqa['data'] = pd.to_datetime(df_covid_iqa['data'], format="%Y-%m-%dT%H:%M:%S")
df_covid_iqa.head()
# print(df_iqa.dtypes)
# print(df_covid_cases_t.dtypes)
#save as csv
df_covid_iqa.to_csv('covid_iqa.csv', index=False)

         data stato  codice_regione denominazione_regione  codice_provincia  \
16 2020-02-24   ITA              15              Campania                63   
24 2020-02-24   ITA               8        Emilia-Romagna                37   
37 2020-02-24   ITA              12                 Lazio                58   
49 2020-02-24   ITA               3             Lombardia                15   
51 2020-02-24   ITA               3             Lombardia                17   

   denominazione_provincia sigla_provincia        lat       long  totale_casi  \
16                  Napoli              NA  40.839566  14.250850            0   
24                 Bologna              BO  44.494367  11.341721            0   
37                    Roma              RM  41.892770  12.483667            0   
49                  Milano              MI  45.466794   9.190347            0   
51                 Brescia              BS  45.539931  10.219103            0   

   note  
16  NaN  
24  NaN  
37  NaN 

C:\Users\g.donatiello\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [1]:
test = pandasql.sqldf(
                    "select denominazione_provincia ,"
                    "count(*) "
                    "from df_covid_iqa "
                    "group by 1"
                    ,locals())
print(test.head(15))

NameError: name 'pandasql' is not defined